In [ ]:
!pip install tensorflow scikit-learn

import tensorflow as tf
import numpy as np
import os
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, Sequential as Seq
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from google.colab import drive
drive.mount('/content/drive')

# ---------------------------
# PATHS
# ---------------------------
train_dir = "/content/drive/MyDrive/database/train"
val_dir = "/content/drive/MyDrive/database/val"
test_dir = "/content/drive/MyDrive/database/test"

# ---------------------------
# LOAD DATASETS
# ---------------------------
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)
val_data = tf.keras.utils.image_dataset_from_directory(
    val_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)
test_data = tf.keras.utils.image_dataset_from_directory(
    test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)

num_classes = len(train_data.class_names)
print("Classes:", train_data.class_names)

# ---------------------------
# NORMALIZATION
# ---------------------------
train_data = train_data.map(lambda x, y: (x / 255.0, y))
val_data = val_data.map(lambda x, y: (x / 255.0, y))
test_data = test_data.map(lambda x, y: (x / 255.0, y))

AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.shuffle(1000).prefetch(AUTOTUNE)
val_data = val_data.prefetch(AUTOTUNE)
test_data = test_data.prefetch(AUTOTUNE)

# ---------------------------
# CLASS WEIGHTS
# ---------------------------
counter = Counter()
for cls in os.listdir(train_dir):
    cls_path = os.path.join(train_dir, cls)
    if os.path.isdir(cls_path):
        counter[int(cls)] += len(os.listdir(cls_path))

print("Class counts:", counter)

classes = np.array(list(counter.keys()))
counts = np.array(list(counter.values()))
total = counts.sum()
class_weights = total / (len(classes) * counts)
class_weights_dict = dict(zip(classes, class_weights))

print("Class weights:", class_weights_dict)

# ---------------------------
# DATA AUGMENTATION
# ---------------------------
data_augmentation = Seq([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1)
])

train_data_aug = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

# ---------------------------
# MODEL BUILD
# ---------------------------
base_model = MobileNetV2(
    weights='imagenet', include_top=False, input_shape=(224, 224, 3)
)
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ---------------------------
# CALLBACKS
model = tf.keras.models.load_model('/content/drive/MyDrive/pest_detection_epoch_10.keras')
checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/pest_detection_epoch_{epoch:02d}.keras',
    save_best_only=False,        # <—— IMPORTANT
    save_weights_only=False,
    verbose=1
)


early_stop = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6
)
model.optimizer.learning_rate.assign(1e-4)

# ---------------------------
# TRAIN TOP LAYERS
# ---------------------------
history =model.fit(
    train_data_aug,
    validation_data=val_data,
    class_weight=class_weights_dict,
    epochs=50,      # or more
    initial_epoch=10,  # this is OK because you already loaded weights
    callbacks=[checkpoint, early_stop, reduce_lr]
)


# ---------------------------
# FINE-TUNE (UNFREEZE LAST LAYERS)
# ---------------------------
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history_fine = model.fit(
    train_data_aug,
    validation_data=val_data,
    epochs=50,
    class_weight=class_weights_dict,
    callbacks=[checkpoint, early_stop, reduce_lr]
)

# ---------------------------
# EVALUATE
# ---------------------------
loss, acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {acc:.4f}")

# ---------------------------
# CLASSIFICATION REPORT
# ---------------------------
y_true = np.concatenate([y for x, y in test_data], axis=0)
y_pred = np.argmax(model.predict(test_data), axis=1)

print(classification_report(
    y_true, y_pred,
    target_names=[f"Class {i}" for i in range(num_classes)]
))

# ---------------------------
# SAVE FINAL MODEL
# ---------------------------
model.save("/content/drive/MyDrive/pest_identifier_model.keras")
print("✅ Saved as pest_identifier_model.keras")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 840.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 134.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 58285 files belonging to 102 classes.
Found 7508 files belonging to 102 classes.
Found 22659 files belonging to 102 classes.
Classes: ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101']
Class counts: Co

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 102)            │        52,326 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,966,182 (11.32 MB)

 Trainable params: 708,198 (2.70 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 11/50
1822/1822 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5253 - loss: 1.7623